In [1]:
from IPython.display import display
import mne
from components.dataset import Dataset_Builder
import pickle
import os
import pandas as pd
import numpy as np
dataset_path = "data"
dataset = Dataset_Builder(dataset_path=dataset_path)\
            .with_sampling_rate(125)\
            .with_marker(start_minute=0, stop_minute=5, segment_second=15)\
            .build()

data, labels, groups = dataset.load_data_all()
data.shape, labels.shape, groups.shape

Found: 55 files
Mean:20.509090909090908, Std:6.149272617066855
Non-stressed:16
Stressed:19
Neutral:20
number of epochs = 20.0


In [2]:
data, labels, groups = dataset.load_data_all()
data.shape, labels.shape, groups.shape

((700, 16, 1875), (700,), (700,))

In [5]:
a,b,c = dataset.load_data('001')
a.shape, b.shape, c.shape

((20, 16, 1875), (20,), (20,))

In [7]:
np.hstack([b,b])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
dataset.wh

array([False,  True, False, False,  True,  True, False,  True, False,
       False, False, False,  True, False, False,  True, False, False,
       False, False,  True, False, False,  True, False,  True, False,
       False,  True, False, False,  True, False,  True, False, False,
       False,  True, False,  True, False, False, False,  True,  True,
       False, False, False,  True, False, False,  True, False,  True,
       False])

In [5]:
data, labels = dataset.load_data('001')
print(data.shape, labels.shape)

(20, 16, 1875) (20,)


In [6]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
dataset.groups

array([0, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 0, 1, 0, 0, 0, 2, 1, 0,
       2, 1, 0, 1, 0, 2, 1, 2, 0, 1, 2, 1, 0, 2, 2, 1, 2, 1, 0, 0, 2, 1,
       1, 0, 2, 2, 1, 2, 2, 1, 0, 1, 2])

In [9]:
print(dataset.start_sample)
print(dataset.stop_sample)
print(dataset.step_sample)
data[:, dataset.start_sample:dataset.stop_sample].shape

7500
22500
1875


(16, 15000)

In [17]:
data = dataset.load_data('001')
# We skip the first marker becase 
data = np.split(data[:, dataset.start_sample:dataset.stop_sample], dataset._expected_epochs_number, axis=1)
print(data[0].shape)
data = [np.expand_dims(data,axis=0) for data in data]
data = np.vstack(data)
data.shape

(16, 1875)


(8, 16, 1875)

In [ ]:
# [np.array(16,125*segment),...] => len(data.shape[1]/segment)
data = [np.expand_dims(data,axis=0) for data in data]
# [np.array(1,16,125*segment),...] => len(data.shape[1]/segment)
data = np.vstack(data)
# (len(data.shape[1]/segment) , 16 , 125*segment )

assert data.shape[0] == dataset._expected_epochs_number, f"data.shape={data.shape}. Expect the first dimension to be {dataset._expected_epochs_number}"

return data